In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.extend([ '../', '../../'])
import torch
import torch.nn as nn
import torch.optim as optim
from data import input_data
import dataloader as dl
from args import Args
from model import Het_En, Classifier, EdgePredictor, Het_classify
from smote import oversample
from train import train_smote, test_smote
import cProfile

# Set device to GPU if available, else use CPU
args = Args()
args.aminer_train()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
# data = input_data(args) 
# torch.save(data, 'data.pt')
data = torch.load('../../data/aminer/am_data.pt')
print(data)

HeteroData(
  a={
    num_nodes=20171,
    y=[20171]
  },
  p={ num_nodes=13250 },
  v={ num_nodes=18 },
  p_title_embed={ x=[13250, 128] },
  p_abstract_embed={ x=[13250, 128] },
  p_net_embed={ x=[13250, 128] },
  p_a_net_embed={ x=[13250, 128] },
  p_p_net_embed={ x=[13250, 128] },
  p_v_net_embed={ x=[13250, 128] },
  a_net_embed={ x=[20171, 128] },
  a_text_embed={ x=[20171, 128] },
  v_net_embed={ x=[18, 128] },
  v_text_embed={ x=[18, 128] },
  (a, walk, a)={ edge_index=[2, 99480] },
  (a, walk, p)={ edge_index=[2, 86444] },
  (a, walk, v)={ edge_index=[2, 31716] },
  (p, walk, a)={ edge_index=[2, 65534] },
  (p, walk, p)={ edge_index=[2, 66174] },
  (p, walk, v)={ edge_index=[2, 20838] },
  (v, walk, a)={ edge_index=[2, 90] },
  (v, walk, p)={ edge_index=[2, 90] },
  (v, walk, v)={ edge_index=[2, 27] }
)


In [3]:
print(data['a','walk','p'].edge_index)
device = args.device
# data = data.to(device)

# Send all x tensors to the device
data['p_title_embed']['x'] = data['p_title_embed']['x'].to(device)
data['p_abstract_embed']['x'] = data['p_abstract_embed']['x'].to(device)
data['p_net_embed']['x'] = data['p_net_embed']['x'].to(device)
data['p_a_net_embed']['x'] = data['p_a_net_embed']['x'].to(device)
data['p_p_net_embed']['x'] = data['p_p_net_embed']['x'].to(device)
data['p_v_net_embed']['x'] = data['p_v_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['v_net_embed']['x'] = data['v_net_embed']['x'].to(device)
data['v_text_embed']['x'] = data['v_text_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)

# Send all edge_index tensors to the device
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 'v']['edge_index'] = data['a', 'walk', 'v']['edge_index'].to(device)
data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 'v']['edge_index'] = data['p', 'walk', 'v']['edge_index'].to(device)
data['v', 'walk', 'a']['edge_index'] = data['v', 'walk', 'a']['edge_index'].to(device)
data['v', 'walk', 'p']['edge_index'] = data['v', 'walk', 'p']['edge_index'].to(device)
data['v', 'walk', 'v']['edge_index'] = data['v', 'walk', 'v']['edge_index'].to(device)

edge_indices = [ data['a', 'walk', 'a'].edge_index, data['a', 'walk', 'p'].edge_index, data['a', 'walk', 'v'].edge_index ]

tensor([[    0,     0,     0,  ..., 20170, 20170, 20170],
        [10523, 10442, 10329,  ...,  4378, 12456,  6612]])


In [4]:
c_train_num = dl.train_num(data['a'].y, args.im_class_num, args.class_sample_num, args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

[50, 20, 50, 50] 170
0 335
1 202
2 299
3 289
train_idx:  [15092, 17180, 550, 13097, 5452, 296, 4172, 16468, 10131, 2512, 11003, 11130, 3232, 15079, 18798, 2741, 13284, 10349, 16654, 18445, 17438, 6410, 15999, 2755, 14143, 19927, 13231, 9930, 15341, 8885, 14312, 9180, 11408, 3842, 12767, 18697, 13810, 14798, 3004, 17253, 13320, 34, 15048, 4254, 19186, 3540, 10244, 9417, 7070, 17533, 4907, 17054, 14560, 5316, 7022, 15443, 4461, 470, 14772, 4270, 12696, 590, 11728, 1260, 7808, 12370, 17532, 7484, 5233, 19706, 11859, 11409, 2441, 5525, 36, 9786, 11858, 13790, 2379, 19661, 657, 6520, 16098, 2158, 8769, 18638, 2455, 11406, 19234, 16623, 20170, 6240, 11027, 19121, 15445, 12640, 15521, 15057, 6050, 4932, 17285, 19813, 15076, 8423, 7082, 15925, 10777, 384, 14340, 1730, 4640, 16812, 4242, 18897, 7441, 19282, 14642, 14867, 1330, 10332, 16232, 2536, 3649, 2073, 17642, 19618, 17328, 10399, 11390, 12019, 15128, 17634, 11838, 10693, 1814, 17881, 18083, 20079, 5856, 10701, 7731, 7043, 10535, 15569, 30

In [5]:
# os_mode = 'up'
# portion = sum(c_train_num)/(4*c_train_num[1])
# if os_mode != 'reweight' and os_mode != 'no':
#     if portion >= 1:
#         args.node_dim[0] = args.A_n + (portion-1)*c_train_num[1] + int((portion - int(portion))*c_train_num[1])
#     else:
#         args.node_dim[0] = args.A_n + int((portion - int(portion))*c_train_num[1])

# args.node_dim[0] += 22

In [6]:
encoder = Het_En(args.embed_dim, args.dropout)
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder_a = EdgePredictor(args.embed_dim)
decoder_p = EdgePredictor(args.embed_dim)
decoder_v = EdgePredictor(args.embed_dim)
decoder_list = [decoder_a, decoder_p, decoder_v]
#print(features.shape)
encoder.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

## Training Part

In [7]:
torch.cuda.empty_cache()
# profiler = cProfile.Profile()
# profiler.enable()
train_smote(data, edge_indices, encoder, classifier, decoder_list, train_idx, val_idx, test_idx, args, os_mode = 'gsm', train_mode = 'preO')
# profiler.disable()

Epoch [1/500], Loss: 1.3959, Accuracy: 0.2604, Edge Accuracy: [0.9997554988434233, 0.9997250077871931, 0.9999641950214554]
Class 0: AUC-ROC- 0.5156, F1 Score- 0.0000; Class 1: AUC-ROC- 0.3911, F1 Score- 0.3493; Class 2: AUC-ROC- 0.4879, F1 Score- 0.0000; Class 3: AUC-ROC- 0.3562, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.4377; Macro-Average F1 Score: 0.0873
Validation Loss: 1.3835, Validation Accuracy: 0.2500, Validation Edge Accuracy: []
Class 0: AUC-ROC- 0.5771, F1 Score- 0.0000; Class 1: AUC-ROC- 0.7485, F1 Score- 0.4000; Class 2: AUC-ROC- 0.6357, F1 Score- 0.0000; Class 3: AUC-ROC- 0.4996, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.6152; Macro-Average F1 Score: 0.1000
Test Loss: 1.3842, Test Accuracy: 0.2525, Test Edge Accuracy: []
Class 0: AUC-ROC- 0.5981, F1 Score- 0.0000; Class 1: AUC-ROC- 0.6929, F1 Score- 0.4000; Class 2: AUC-ROC- 0.6645, F1 Score- 0.0000; Class 3: AUC-ROC- 0.4121, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5919; Macro-Average F1 Score: 0.1000

Epoch [2/500],

In [8]:
# profiler.print_stats(sort='cumulative')  # Print results to the console
# # Or save to a file
# with open('profile_results.txt', 'w') as f:
#     profiler.dump_stats(f)

In [9]:
test_smote(data, edge_indices, encoder, classifier, decoder_list, test_idx, args = args, dataset = 'Test', os_mode = 'no', train_mode = 'preO')

Test Loss: 0.8555, Test Accuracy: 0.9700, Test Edge Accuracy: []
Class 0: AUC-ROC- 1.0000, F1 Score- 0.9950; Class 1: AUC-ROC- 0.9999, F1 Score- 0.9851; Class 2: AUC-ROC- 0.9977, F1 Score- 0.9296; Class 3: AUC-ROC- 0.9986, F1 Score- 0.9697; Macro-Average AUC-ROC: 0.9991; Macro-Average F1 Score: 0.9698


(0.97, 0.9990583333333334, 0.9698309839308082)